In [1]:
!pip install tweepy

In [2]:
!pip install textblob

# importation des bibliothéque

In [3]:
from tweepy import API #permet de se connecter en mode développeur
from tweepy import Cursor #parcourir le contenu
from tweepy.streaming import StreamListener #suivre en mode real time
from tweepy import OAuthHandler #authentification
from tweepy import Stream #data tracking

from textblob import TextBlob #analyse de sentiment
 
import twitter_credentials #twitter user data

import matplotlib.pyplot as plt #afficher une courbe
import numpy as np #lire les valeur en chiffre
import pandas as pd #lire la valeur as dataset
import re #reader


# authentification

In [5]:
class TwitterAuthenticator():

    def authenticate_twitter_app(self):
        auth = OAuthHandler(twitter_credentials.CONSUMER_KEY, twitter_credentials.CONSUMER_SECRET)
        auth.set_access_token(twitter_credentials.ACCESS_TOKEN, twitter_credentials.ACCESS_TOKEN_SECRET)
        return auth


# definition de twitter client

In [6]:

# # # # TWITTER CLIENT # # # #
class TwitterClient():
    def __init__(self, twitter_user=None):
        self.auth = TwitterAuthenticator().authenticate_twitter_app()
        self.twitter_client = API(self.auth)

        self.twitter_user = twitter_user

    def get_twitter_client_api(self):
        return self.twitter_client

    def get_user_timeline_tweets(self, num_tweets):
        tweets = []
        for tweet in Cursor(self.twitter_client.user_timeline, id=self.twitter_user).items(num_tweets):
            tweets.append(tweet)
        return tweets

    def get_friend_list(self, num_friends):
        friend_list = []
        for friend in Cursor(self.twitter_client.friends, id=self.twitter_user).items(num_friends):
            friend_list.append(friend)
        return friend_list

    def get_home_timeline_tweets(self, num_tweets):
        home_timeline_tweets = []
        for tweet in Cursor(self.twitter_client.home_timeline, id=self.twitter_user).items(num_tweets):
            home_timeline_tweets.append(tweet)
        return home_timeline_tweets


twitter streamer (hashtags, photo data)


In [7]:

# # # # TWITTER STREAMER # # # #
class TwitterStreamer():
   
    def __init__(self):
        self.twitter_autenticator = TwitterAuthenticator()    

    def stream_tweets(self, fetched_tweets_filename, hash_tag_list):
        # This handles Twitter authetification and the connection to Twitter Streaming API
        listener = TwitterListener(fetched_tweets_filename)
        auth = self.twitter_autenticator.authenticate_twitter_app() 
        stream = Stream(auth, listener)

        # This line filter Twitter Streams to capture data by the keywords: 
        stream.filter(track=hash_tag_list)

In [8]:
# # # # TWITTER STREAM LISTENER # # # #
class TwitterListener(StreamListener):
  
    def __init__(self, fetched_tweets_filename):
        self.fetched_tweets_filename = fetched_tweets_filename

    def on_data(self, data):
        try:
            print(data)
            with open(self.fetched_tweets_filename, 'a') as tf:
                tf.write(data)
            return True
        except BaseException as e:
            print("Error on_data %s" % str(e))
        return True
          
    def on_error(self, status):
        if status == 420:
            # Returning False on_data method in case rate limit occurs.
            return False
        print(status)

In [9]:

class TweetAnalyzer():
    """
    Functionality for analyzing and categorizing content from tweets.
    """

    def clean_tweet(self, tweet):
        return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

    def analyze_sentiment(self, tweet):
        analysis = TextBlob(self.clean_tweet(tweet))
        
        if analysis.sentiment.polarity > 0:
            return "good"
        elif analysis.sentiment.polarity == 0:
            return "neutral not so good"
        else:
            return " very bad/angry"

    def tweets_to_data_frame(self, tweets):
        df = pd.DataFrame(data=[tweet.text for tweet in tweets], columns=['tweets'])

        
        #df['date'] = np.array([tweet.created_at for tweet in tweets])


        return df


In [11]:
if __name__ == '__main__':

    twitter_client = TwitterClient()
    tweet_analyzer = TweetAnalyzer()

    api = twitter_client.get_twitter_client_api()

    tweets = api.user_timeline(screen_name="BarackObama", count=200)

    df = tweet_analyzer.tweets_to_data_frame(tweets)
    df['sentiment'] = np.array([tweet_analyzer.analyze_sentiment(tweet) for tweet in df['tweets']])

    print(df.head(20))
    
    

                                               tweets            sentiment
0   While we continue to wait for a coherent natio...                 good
1   We've all had to adapt to cope with a pandemic...                 good
2   Parents, you deserve a break – Michelle’s got ...                 good
3   Proud of the leaders in my hometown of Chicago...                 good
4   When I think of people I’ve counted on over th...                 good
5   Democracy depends on an informed citizenry and...                 good
6   For all of us who love this country and are wi...                 good
7   I’m proud to endorse my friend @JoeBiden for P...                 good
8   Dreamers have contributed so much to our count...                 good
9   We can't deny that racial and socioeconomic fa...  neutral not so good
10  Although our celebrations may look different t...  neutral not so good
11  Let’s not use the tragedy of a pandemic to com...  neutral not so good
12  Everyone should have 